# Creación de Datos Entrega 2

En este archivo se crearán los datos que se utilizarán para simular el modelo del proyecto. Esta simulación se resolverá con la ayuda de pygurobi, una librería open source de python.

En este archivo jupyter se hará uso de la librería *pandas*(no necesita que se descargue) para darle valores a los parámetros necesarios para la simulación. Estos serán guardados en arhivos .csv, los cuales serán después leídos desde el archivo `main.py`.

Cabe mencionar que se intentó utilizar datos de empresas reales, pero estas no estuvieron dispuestas a colaborar por temas de confidencialidad empresarial.\\

Se simularán **1000 productos** para un espacio de **28 días**, lo que serían 4 semanas completas comenzando desde el lunes.

## Parámetros

Existen básicamente dos tipos de parámetros. Los primeros son los que tienen como sufijo p(producto) y t(días), y otros que tienen como sufijo solo p. Por lo tanto, se creará un archivo .csv para cada uno de estos pares de parametros.

Acá solo nos faltarían los parametros V<sub>q</sub> y PR. Dado que son solo 4 datos que se deben simular, se calcularán los valores aleatoriamente en rangos que nos parezcan razonables, para que luego se ocupen estos mismos valores en el archivo `main.py`

### Parámetros con sufijo <sub>pt</sub>

Comenzaremos por crear un DataFrame que tenga una fila para cada par (p,t)

In [70]:
import pandas
import random
df_pt = pandas.DataFrame({"p": [x for x in range(1,101)]*28}).sort_values("p")
df_pt.insert(1, "t", [x for x in range(1,29)]*100)
print(df_pt)

        p   t
0       1   1
2700    1   2
300     1   3
400     1   4
2600    1   5
...   ...  ..
2299  100  24
2399  100  25
2499  100  26
1299  100  27
2799  100  28

[2800 rows x 2 columns]


#### F<sub>pt</sub>

Este parametro indica el costo fijo por hacer un pedido del producto *p* en el día *t*.
Asumiremos que este costo solo varía por día de semana. Así, en el día 1 (primer lunes) y en el día 8 (segundo lunes) el costo fijo por pedir el producto *p* será el mismo. Se utiliza este supuesto porque en la vida real es así, pues las semanas son muy parecidas entre ellas en cuanto a temas de logística

Para cada producto *p* se comenzará calculando un costo fijo representativo, cuyo valor variará entre $20.000 y $30.000 para cada producto *p*. Para hacerlo más realista aun, los precios
de los costos fijos del producto *p* no podrán variar más de un 10% del costo fijo representativo.

In [71]:
costos_fijos = list()
for p in range(100):
    valor_representativo_p = random.randint(20000, 30000)
    costos_fijos_p = list()
    for t in range(7):
        costos_fijos_p.append(random.randint(int(valor_representativo_p*0.9), int(valor_representativo_p*1.1)))
    costos_fijos.extend(costos_fijos_p*4)
df_pt.insert(2, "F", costos_fijos)

### n<sub>pt</sub>

Este parametro indica el costo de una unidad de *p* en el día *t*. Nuevamente asumiremos que para cada día de la semana, el producto va valer lo mismo en distintas semanas.

El valor representativo de este parametro variará entre $1.000 y $5.000. Luego, el valor para cada día de la semana no podrá variar más de un 10% del valor representativo.

In [72]:
costos_unitarios = list()
for p in range(100):
    valor_representativo_p = random.randint(1000, 5000)
    costos_unitarios_p = list()
    for t in range(7):
        costos_unitarios_p.append(random.randint(int(valor_representativo_p*0.9), int(valor_representativo_p*1.1)))
    costos_unitarios.extend(costos_unitarios_p*4)
df_pt.insert(3, "n", costos_unitarios)

### b<sub>pt</sub>

Este parametro indica el costo de guardar en bodega una unidad del producto *p* desde el día *t* hasta *t+1*. Nuevamente asumiremos que para cada día de la semana, el costo de bodegaje va se el mismo en distintas semanas.

El valor representativo de este parametro variará entre $100 y $500. Luego, el valor para cada día de la semana no podrá variar más de un 10% del valor representativo.

In [73]:
costos_bodegaje = list()
for p in range(100):
    valor_representativo_p = random.randint(100, 500)
    costos_bodegaje_p = list()
    for t in range(7):
        costos_bodegaje_p.append(random.randint(int(valor_representativo_p*0.9), int(valor_representativo_p*1.1)))
    costos_bodegaje.extend(costos_bodegaje_p*4)
df_pt.insert(4, "b", costos_bodegaje)

### d<sub>pt</sub>

Este parametro indica la demanda del producto *p* en el día *t*. En este caso no asumiremos que la demanda semana a semana es igual, porque pueden haber sucesos como fiestas nacionales que hacen variar bruscamente la demanda por ciertos bienes.

El valor representativo de este parametro variará entre 10 y 100 unidades diarias. No existe restricción de variación

In [74]:
df_pt.insert(5, "d", [random.randint(10,100) for x in range(2800)])

## Parámetros con sufijo <sub>p</sub>

In [87]:
df_p = pandas.DataFrame({"p": [x for x in range(1,101)]}).sort_values("p")

### v<sub>p</sub>

In [88]:
df_p.insert(1, "v", [random.randint(1,10) for x in range(100)])

## maxc<sub>p</sub>

In [89]:
df_p.insert(2, "maxc", [random.randint(100,1000) for x in range(100)])

## minc<sub>p</sub>

In [90]:
df_p.insert(3, "minc", [random.randint(1,10) for x in range(100)])

## z<sub>p</sub>

In [91]:
precios_ventas = list()
for p in range(1,101):
    precios_compra_p = df_pt[df_pt["p"] == p]
    precio_venta_p = random.randint(int(precios_compra_p["n"].max()*1.1), int(precios_compra_p["n"].max()*1.3))
    precios_ventas.append(precio_venta_p)
df_p.insert(4, "z", precios_ventas)

## exp<sub>p</sub>

In [92]:
df_p.insert(5, "exp", [random.randint(2,5) for x in range(100)])

## u<sub>p</sub>

In [93]:
df_p.insert(6, "u", [random.randint(1,100) for x in range(100)])

### Extras

## Pasar los datos a CSV